# 14. 합성곱 신경망을 사용한 컴퓨터 비전

In [1]:
# 공통 모듈 임포트
import numpy as np
import os

# 깔금한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)    
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [2]:
import tensorflow as tf

## 14.1 시각 피질 구조

## 14.2 합성곱 층

### 14.2.1 필터

### 14.2.2 여러 가지 특성 맵 쌓기

### 14.2.3 케라스로 합성곱 층 구현하기

In [19]:
from sklearn.datasets import load_sample_images

images = load_sample_images()['images']
images = tf.keras.layers.CenterCrop(height=70, width=120)(images)
images = tf.keras.layers.Rescaling(scale = 1 / 255)(images)

In [21]:
images.shape

TensorShape([2, 70, 120, 3])

In [22]:
conv_layer = tf.keras.layers.Conv2D(filters=32, kernel_size=7)
fmaps = conv_layer(images)

In [23]:
fmaps.shape

TensorShape([2, 64, 114, 32])

In [25]:
conv_layer = tf.keras.layers.Conv2D(filters=32, kernel_size=7, padding='same')
fmaps = conv_layer(images)
fmaps.shape

TensorShape([2, 70, 120, 32])

In [26]:
kernels, biases = conv_layer.get_weights()
kernels.shape

(7, 7, 3, 32)

In [27]:
biases.shape

(32,)

### 14.2.4 메모리 요구 사항

## 14.3 풀링 층

## 14.4 케라스로 풀링 층 구현하기

In [28]:
max_pool = tf.keras.layers.MaxPool2D(pool_size=2)

In [29]:
class DepthPool(tf.keras.layers.Layer):
    def __init__(self, pool_size = 2, **kwargs):
        super().__init__(**kwargs)
        self.pool_size = pool_size
    
    def call(self, inputs):
        shape = tf.shape(inputs)
        groups = shape[-1] // self.pool_size # shape[-1] = 채널 개수
        new_shape = tf.concat([shape[:-1], [groups, self.pool_size]], axis=0)
        return tf.reduce_max(tf.reshape(inputs, new_shape), axis = -1)

In [30]:
global_avg_pool = tf.keras.layers.GlobalAvgPool2D()

In [31]:
global_avg_pool = tf.keras.layers.Lambda(lambda X: tf.reduce_mean(X, axis=[1, 2]))

In [32]:
global_avg_pool(images)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.64338803, 0.5971821 , 0.58250374],
       [0.76307476, 0.26010975, 0.10848834]], dtype=float32)>

## 14.5 CNN 구조

In [5]:
from functools import partial

DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size = 3, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')

model = tf.keras.Sequential([
    DefaultConv2D(filters = 64, kernel_size = 7, input_shape = [28, 28, 1]),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units = 128, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units = 64, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.5),   
    tf.keras.layers.Dense(units=10, activation='softmax')
])

2023-10-12 23:51:25.853292: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-10-12 23:51:25.853437: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-10-12 23:51:25.853447: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-10-12 23:51:25.853817: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-12 23:51:25.854376: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## 14.6 케라스로 ResNet-34 CNN 구현하기

In [6]:
DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size = 3, strides=1, padding='same', kernel_initializer = 'he_normal', use_bias=False)

class ResidualUnit(tf.keras.layers.Layer):
    def __init__(self, filters, strides = 1, activation = 'relu', **kwargs):
        super().__init__(**kwargs)
        self.activation = tf.keras.activations.get(activation)

        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            tf.keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            tf.keras.layers.BatchNormalization()
        ]

        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                tf.keras.layers.BatchNormalization()
            ]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)

        return self.activation(Z + skip_Z)

In [8]:
model = tf.keras.Sequential([
    DefaultConv2D(64, kernel_size=7, strides=2, input_shape=[224, 224, 3]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')
])

prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters

model.add(tf.keras.layers.GlobalAvgPool2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation='softmax'))

## 14.7 케라스의 사전 훈련 모델 사용하기

In [10]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

model = tf.keras.applications.resnet50.ResNet50(weights='imagenet')

102967424/102967424 [==============================] - 18s 0us/step


In [12]:
from sklearn.datasets import load_sample_images

images = load_sample_images()['images']
images_resized = tf.keras.layers.Resizing(height=224, width=224, crop_to_aspect_ratio=True)(images)

In [13]:
inputs = tf.keras.applications.resnet50.preprocess_input(images_resized)

In [18]:
Y_proba = model.predict(inputs)
Y_proba.shape

1/1 [==============================] - 0s 92ms/step


(2, 1000)

In [19]:
top_K = tf.keras.applications.resnet50.decode_predictions(Y_proba, top=3)
for image_index in range(len(images)):
    print(f"이미지 #{image_index}")
    for class_id, name, y_proba in top_K[image_index]:
        print(f"  {class_id} - {name:12s} {y_proba:.2%}")

이미지 #0
  n03877845 - palace       54.69%
  n03781244 - monastery    24.71%
  n02825657 - bell_cote    18.55%
이미지 #1
  n04522168 - vase         32.67%
  n11939491 - daisy        17.82%
  n03530642 - honeycomb    12.04%


## 14.8 사전 훈련된 모델을 사용한 전이 학습

In [21]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
dataset_size = info.splits["train"].num_examples
class_names = info.features["label"].names
n_classes = info.features["label"].num_classes

2023-10-13 00:03:48.863129: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /Users/oneash/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [27]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    'tf_flowers',
    split=['train[:10%]', 'train[10%:25%]', 'train[25%:]'],
    as_supervised=True
)

In [28]:
batch_size = 32
preprocess = tf.keras.Sequential([
    tf.keras.layers.Resizing(height=224, width=224, crop_to_aspect_ratio=True),
    tf.keras.layers.Lambda(tf.keras.applications.xception.preprocess_input)
])
train_set = train_set_raw.map(lambda X, y: (preprocess(X), y))
train_set = train_set.shuffle(1000, seed=42).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(lambda X, y: (preprocess(X), y)).batch(batch_size)
test_set = test_set_raw.map(lambda X, y: (preprocess(X), y)).batch(batch_size)

In [34]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(mode="horizontal", seed=42),
    tf.keras.layers.RandomRotation(factor=0.05, seed=42),
    tf.keras.layers.RandomContrast(factor=0.2, seed=42)
])

In [35]:
base_model = tf.keras.applications.xception.Xception(weights='imagenet', include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(n_classes, activation='softmax')(avg)
model = tf.keras.Model(inputs=base_model.input, outputs = output)

83683744/83683744 [==============================] - 18s 0us/step


In [36]:
for layer in base_model.layers:
    layer.trainable = False

In [38]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=3)

Epoch 1/3


2023-10-13 00:24:47.269708: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


86/86 [==============================] - ETA: 0s - loss: 0.2586 - accuracy: 0.9197

2023-10-13 00:25:37.622743: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


86/86 [==============================] - 63s 714ms/step - loss: 0.2586 - accuracy: 0.9197 - val_loss: 0.4829 - val_accuracy: 0.8766
Epoch 2/3
86/86 [==============================] - 67s 761ms/step - loss: 0.1665 - accuracy: 0.9484 - val_loss: 0.6745 - val_accuracy: 0.8475
Epoch 3/3
86/86 [==============================] - 68s 778ms/step - loss: 0.1174 - accuracy: 0.9589 - val_loss: 0.4972 - val_accuracy: 0.8730


In [39]:
for layer in base_model.layers[56:]:
    layer.trainable = True

In [40]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=10)

Epoch 1/10


2023-10-13 00:28:07.561039: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 7/86 [=>............................] - ETA: 1:48 - loss: 0.4581 - accuracy: 0.8393

## 14.9 분류와 위치 추정

In [ ]:
base_model = tf.keras.applications.xception.Xception(weights = 'imagenet', include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
class_output = tf.keras.layers.Dense(n_classes, activation="softmax")(avg)
loc_output = tf.keras.layers.Dense(4)(avg)
model = tf.keras.Model(inputs=base_model.input, outputs = [class_output, loc_output])
model.compile(loss=['sparse_categorical_crossentropy', 'mse'],
              loss_weights = [0.8, 0.2],
              optimizer = optimizer, metrics = ['accuracy'])